# Sibyl evaluation results
Results of the evaluation by Sibyl team. Data available here: https://github.com/sibyl-team/ViraTrace-example

In [29]:
import numpy as np
import pandas as pd
import tqdm

from multisim_model_evaluation import MultisimModelEvaluation

In [30]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [65]:
contacts = pd.read_csv("contacts_t_12.csv.gz",index_col=0)

/home/ivan/projects/courier/venv-courier/lib/python3.8/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [66]:
observations = pd.read_csv("observ_t_12.csv.gz",index_col=0)

### Description

We have 12 days of contacts and a variable number of observations up until day 11

We want to obtain the risk prediction for the nodes on day 12

In [67]:
observations

,i,state,t
0,3032,1,7
1,6865,0,7
2,2868,0,7
3,518,0,7
4,7375,0,7
...,...,...,...
47,2207,0,11
48,5215,0,11
49,6637,0,11
50,5255,0,11


In [80]:
contacts

,a,b,t
0,0,3,0
1,0,2,0
2,0,1,0
3,0,1540,0
4,0,7583,0
...,...,...,...
1320789,9999,2726,12
1320790,9999,7050,12
1320791,9999,6990,12
1320792,9999,9242,12


In [69]:
contacts.rename(columns={'i': 'a', 'j': 'b'}, inplace=True)

In [70]:
observations.rename(columns={'i': 'a'}, inplace=True)

In [94]:
N = max(contacts['a'])+1
n_days = max(contacts['t'])
INFECTION_P = 0.02
INFECTED_P = 1/N
RECOVERY_T = 50

N_SIMS = 1000

edge_batch_gen = ((t, contacts[contacts.t == t]) for t in range(0, contacts.t.max()))

In [95]:
model = MultisimModelEvaluation(N, n_days, N_SIMS, edge_batch_gen, observations,
                                INFECTION_P, INFECTED_P, RECOVERY_T, None, recovery_dist='geometric',
                                directed_edges=True, tqdm=tqdm.tqdm)

In [96]:
model.run_sim()

Starting simulation...: 100%|██████████| 12/12 [00:34<00:00,  2.87s/it]


In [99]:
risk = pd.Series(model.I.sum(axis=1)).sort_values(ascending=False)
risk = (risk / risk.max())
risk.head(20)

9438    1.000
8182    1.000
1085    0.979
5647    0.977
1       0.976
8345    0.968
1839    0.948
6688    0.947
6118    0.942
4698    0.940
3032    0.929
1016    0.114
5210    0.104
6690    0.094
3       0.086
4699    0.084
3033    0.083
5232    0.083
6692    0.082
2       0.081
dtype: float64

In [100]:
risk.to_csv("nodes_risk_original_model.csv")